In [1]:
# import cv2
# import matplotlib.pyplot as plt

# # Specify the path to your input image
# input_image_path = "/kaggle/input/imagetes/video_20230131_145825_00029_jpg.rf.4e4709b4880012206f6ecd0a4f26465b.jpg"  # Update with your image path

# # Run the model prediction on the input image
# # You can adjust the confidence threshold (e.g., conf=0.25) as needed
# results = model.predict(source=input_image_path, conf=0.25, show=False)

# # The 'results' object is a list; for a single image, we access the first element.
# result = results[0]

# # Get the annotated image (with bounding boxes and labels) from the result.
# # The 'plot()' method returns the annotated image.
# annotated_image = result.plot()

# # Display the annotated image using matplotlib
# plt.figure(figsize=(10, 10))
# # OpenCV uses BGR by default, so we convert it to RGB for correct display
# plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
# plt.axis("off")
# plt.title("Predicted Output")
# plt.show()

# # Optionally, print the detection details such as bounding boxes, classes, and confidence scores.
# print("Detection details:")
# print(result.boxes)


In [2]:
# # Uncomment these if you're running in a new environment to ensure packages are installed:
# # !pip install ultralytics
# # !pip install ray==2.0.0

# # -------------------------------
# # 1. Patch Ray to Avoid _get_session Error
# # -------------------------------
# try:
#     import ray
#     try:
#         # Try to import the session module from ray.train._internal
#         import ray.train._internal.session as session
#         # If the _get_session attribute does not exist, add a dummy version that returns None.
#         if not hasattr(session, "_get_session"):
#             session._get_session = lambda: None
#             print("Ray patch applied: _get_session has been set to a dummy function.")
#         else:
#             print("ray.train._internal.session already has _get_session.")
#     except Exception as inner_error:
#         print("Could not patch ray.train._internal.session:", inner_error)
# except ImportError:
#     print("Ray is not installed. Skipping ray patching.")

# # -------------------------------
# # 2. Imports and Dataset Setup
# # -------------------------------
# import os
# import torch
# import yaml
# from pathlib import Path
# from ultralytics import YOLO

# # Set the dataset path (adjust as necessary; this example is for Kaggle)
# DATASET_PATH = "/kaggle/input/wastet"

# # Create a data.yaml configuration file with absolute paths
# data_yaml = {
#     'train': f"{DATASET_PATH}/train/images",
#     'val': f"{DATASET_PATH}/valid/images",
#     'test': f"{DATASET_PATH}/test/images",
#     'nc': 4,
#     'names': ['cap', 'paper', 'plastic', 'shell']
# }

# # Write the data.yaml file
# with open('data.yaml', 'w') as f:
#     yaml.dump(data_yaml, f)

# print("Data configuration saved in data.yaml:")
# with open('data.yaml', 'r') as f:
#     print(f.read())

# # Verify dataset paths
# print("Verifying paths:")
# print("Train path exists:", os.path.exists(data_yaml['train']))
# print("Validation path exists:", os.path.exists(data_yaml['val']))
# print("Test path exists:", os.path.exists(data_yaml['test']))

# # Check for GPU availability
# print("\nGPU Available:", torch.cuda.is_available())
# if torch.cuda.is_available():
#     print("GPU Name:", torch.cuda.get_device_name(0))

# # -------------------------------
# # 3. Initialize and Train YOLOv8 Model
# # -------------------------------
# # Initialize YOLOv8 model with a pretrained weight file (you can use 'yolov8n.pt' for fine-tuning)
# model = YOLO('yolov8n.pt')

# # Training configuration
# train_args = {
#     'data': str(Path('data.yaml').absolute()),  # Use absolute path to data.yaml
#     'epochs': 50,
#     'imgsz': 640,
#     'batch': 8,
#     'patience': 20,
#     'device': 0,         # Use GPU if available (0 indicates first GPU)
#     'workers': 2,
#     'save': True,
#     'cache': True,
#     'project': 'waste_detection',
#     'name': 'training_run',
#     'exist_ok': True
# }

# print("\nStarting training...")
# # Train the model with keyword arguments. The patched Ray callback should now be safe.
# model.train(**train_args)
# print("Training completed successfully!")

# # -------------------------------
# # 4. Save the Trained Model
# # -------------------------------
# model_save_path = 'waste_detection_best.pt'
# model.save(model_save_path)
# print(f"\nModel saved successfully as {model_save_path}")

# # Verify that the model file exists by listing its details
# print("\nSaved model file details:")
# !ls -lh {model_save_path}


In [3]:
# import pandas as pd
# import matplotlib.pyplot as plt

# # Load training results from CSV
# results_df = pd.read_csv('/kaggle/working/waste_detection/training_run/results.csv')

# # Print available columns to verify the structure
# print("Columns in results.csv:", results_df.columns.tolist())

# # Create a figure to plot the metrics
# plt.figure(figsize=(10, 6))

# # Plot Precision, Recall, mAP50, and mAP50-95 over epochs using the corresponding columns
# plt.plot(results_df['epoch'], results_df['metrics/precision(B)'], label='Precision (B)', marker='o')
# plt.plot(results_df['epoch'], results_df['metrics/recall(B)'], label='Recall (B)', marker='s')
# plt.plot(results_df['epoch'], results_df['metrics/mAP50(B)'], label='mAP50 (B)', marker='^')
# plt.plot(results_df['epoch'], results_df['metrics/mAP50-95(B)'], label='mAP50-95 (B)', marker='d')

# # Customize the plot
# plt.xlabel('Epoch')
# plt.ylabel('Metric Value')
# plt.title('Training Metrics Over Epochs for Waste Detection')
# plt.legend()
# plt.grid(True)
# plt.show()
